# Pandas

While `numpy` arrays are fast, they lack useful annotation regarding what the data actually represents.  
A commonly used library to account for that is `pandas`, which introduces two new data types: `Series` and `DataFrame`.  
You can think a `Series` as an annotated vector (with an index) and a `DataFrame` as an annotated matrix (with an index and columns).  

In [ ]:
from __future__ import annotations

from typing import Any, cast

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.typing import NDArray
from pathlib import Path


def test(x: Any, expected: Any) -> None:
    if isinstance(x, np.ndarray) and isinstance(expected, np.ndarray):
        if not np.array_equal(x, expected):
            raise AssertionError(f"Expected {expected}, got {x}")
    elif isinstance(x, pd.DataFrame) and isinstance(expected, pd.DataFrame):
        if not x.equals(expected):            
            raise AssertionError(f"Expected {expected}, got {x}")
    elif isinstance(x, pd.Series) and isinstance(expected, pd.Series):
        if not x.equals(expected):            
            raise AssertionError(f"Expected {expected}, got {x}")
    else:
        if x != expected:
            raise AssertionError(f"Expected {expected}, got {x}")
    print("Test passed")


## Series construction

Series can be constructed in a variety of ways.  
If you pass an array / list, by default an integer index will be added.  

In [ ]:
versicolor = pd.Series([4.7, 3.2, 1.3, 0.2])
versicolor


However, you can insert a custom index in order to better communicate what the data describes.

In [ ]:
index = [
    "sepal length (cm)",
    "sepal width (cm)",
    "petal length (cm)",
    "petal width (cm)",
]

versicolor.index = pd.Index(index)
versicolor


You can also directly add the index when initializing the object.

In [ ]:
virginica = pd.Series([5.1, 3.5, 1.4, 0.2], index=index)
virginica

Or, my preferred way, you can simply use a dictionary and let `pandas` use the dictionary keys as the index.

In [ ]:
setosa = pd.Series(
    {
        "sepal length (cm)": 4.9,
        "sepal width (cm)": 3.0,
        "petal length (cm)": 1.4,
        "petal width (cm)": 0.2,
    }
)
setosa


## DataFrame construction

`DataFrame`s can be built analogously.  
Since I didn't want to repeat all the possibilities, I just used the `Series` we had before as dictionary values.  
You can also create nested dictionaries, or a 2d matrix with the keyword arguments `index` and `columns` respectively for this. 

In [ ]:
iris = pd.DataFrame(
    {
        "virginica": virginica,
        "setosa": setosa,
        "versicolor": versicolor,
    }
)
iris


## Quick stats: min, max, mean, std

Since pandas was built with statistical analysis in mind you can very quickly do basic statistics on the data sets using `.describe`

In [ ]:
iris.describe()

If you are only interested in portions of this, you can call each of the functions separately.

In [ ]:
print("Mean sepal length: {:.2f}".format(iris.loc["sepal length (cm)"].mean()))
print("Min sepal length : {:.2f}".format(iris.loc["sepal length (cm)"].min()))
print("Max sepal length : {:.2f}".format(iris.loc["sepal length (cm)"].max()))
print("sepal length std : {:.2f}".format(iris.loc["sepal length (cm)"].std()))

## Indexing

Note that `pandas` uses `.loc[idx]` to index, instead of just plain `[idx]`.  


In [ ]:
virginica.loc["sepal length (cm)"]



This is because there is a second way using `.iloc[idx]` if you want to fall back to using integer indices to access your data.   



In [ ]:
virginica.iloc[0]


As `DataFrame`s have two dimensions, indexing is done using `.loc[index, column]`.

In [ ]:
iris.loc["sepal length (cm)", "setosa"]


If all columns are required, you can leave out the second argument.


In [ ]:
iris.loc["sepal length (cm)"]




And if all rows required, the usual slicing syntax works


In [ ]:
iris.loc[:, "setosa"]


If you ever need to view or access the respective indexes directly, you can use `.index` and `.columns` (on `DataFrame`s) for that.


In [ ]:
print(setosa.index)  # Index(['sepal length (cm)', 'sepal width (cm)', ...
print(iris.index)    # Index(['sepal length (cm)', 'sepal width (cm)', ...
print(iris.columns)  # Index(['virginica', 'setosa', ...



**Note**

You *can* just use `[idx]` to index into both `Series` and `DataFrames`.  
Do note though, that in that case in a `Series` you index into the index, and in a `DataFrame` you index into the *columns*, so it is easy to mix things up.


## Filtering

You can use boolean filters just like with numpy arrays to select a portion of the data

In [ ]:
sepal_length = iris.loc["sepal length (cm)"]
sepal_length[sepal_length < 5]

### Exercise: filtering

Given the `DataFrame` of Arabidopsis thaliana mutants below, select only those for which the score is between 15 and 20.

In [ ]:
mutants = pd.DataFrame(
    {
        "name": [
            "wt",
            "col0",
            "col1",
            "ara",
            "core",
            "col17",
            "col18",
            "over",
            "mut",
            "mu5",
        ],
        "score": [12.5, 9, 16.5, np.nan, 9, 20, 14.5, np.nan, 8, 19],
        "region": [1, 3, 2, 3, 2, 3, 1, 1, 2, 1],
        "qualify": ["yes", "no", "yes", "no", "no", "yes", "yes", "no", "no", "yes"],
    }
)

expected = pd.DataFrame(
    {
        "name": {2: "col1", 5: "col17", 9: "mu5"},
        "score": {2: 16.5, 5: 20.0, 9: 19.0},
        "region": {2: 2, 5: 3, 9: 1},
        "qualify": {2: "yes", 5: "yes", 9: "yes"},
    }
)

filtered = ...

test(filtered, expected)

## Plotting

`Pandas` has a quick way to create plots from `DataFrame`s and `Series`, using the `.plot` method.  
The method takes a few optional parameters, the most interesting being `kind`, with which you can create different plot kinds.  
Here is a list of the supported plots:

| name    | description                    |
|---------|--------------------------------|
| line    | line plot (default)            |
| bar     | vertical bar plot              |
| barh    | horizontal bar plot            |
| hist    | histogram                      |
| box     | boxplot                        |
| kde     | Kernel Density Estimation plot |
| area    | area plot                      |
| pie     | pie plot                       |
| scatter | scatter plot (DataFrame only)  |
| hexbin  | hexbin plot (DataFrame only)   |

For a complete explanation of all the options, refer to the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html)

In [ ]:
ax = iris.T.plot()

In [ ]:
ax = iris.T.plot(kind="bar")

In [ ]:
ax = iris.T.plot(kind="bar", stacked=True)

In [ ]:
ax = iris.plot(kind="area")

## Hands-on learning: by-catch data set

You are now ready to work with your first real data set!  
We will start below by loading subset of the turtle data you will be working with later.  
We are using the `read_csv` function to load the `bycatch.csv`.  
The `index_col=0` argument sets the first column as the index of the data frame.  

After that the `rename` call changes one of the column names, as it contains some trailing whitespace, which otherwise will have a high chance of tripping us over later.

Lastly, we convert the caught date into a Python `datetime` object, that will allow us to more easily manipulate this data later on.  

Feel free to comment out different parts of this to see what exactly they are doing. 

In [ ]:
df1 = (
    pd.read_csv(Path("data") / "bycatch.csv", index_col=0)
    .rename(columns={"Caught date ": "Caught date"})
)
df1["Caught date"] = pd.to_datetime(df1["Caught date"], format=r"%m/%d/%y")


Use the `.head()` method to inspect the first few rows of the data

In [ ]:
df1.head()

### Exercise: caught by species

Select the `"Species"` column of the data frame and use the `.value_counts` function to see how many individuals were caught by species.

Note that this returns a `Series` object.  
Use the `plot` function on this object to create a bar plot.

### Exercise: length distribution

Plot a histogram of the curved carapace length (ccl) of all turtles.  
What is the most common size?  
What kind of distribution do you see?  

### Exercise: weight distribution

Plot a histrogram of the weight of all turtles.  
What is the most common weight?  
What kind of distribution do you see?  

### Exercise: length vs weight

Plot the length (ccl) against the weight of the turtles as a *scatter plot*.  
Do you have any idea what could cause the shape you observe?  

### Exercise: times each turtle got caught

Next we want to know how often each turtle got caught.  
This will help us to assess how much temporal information we can get out of the data.  


Plot a histogram of how often each turtle got caught.  

Hint: using `value_counts` you can count all the `Turtle ID`s.  



### Exercise: place of capture

Now we want to know *where* the turtles were captured.  

Plot a histrogram of how often the turtles were caught at each capture site.  

## Groupby

One very powerful technique to work on data is to split it into different groups, applying a function on each group and then combining the results of each operation again.  
Pandas has a function called `groupby` which can be used for this.  

We are going to work with the dates of capture next, so we need a very quick introduction into how `Datetime` objects work.  

In order to get all `Datetime` objects in a column, we need the `.dt` method.  
This will allow us to call `Datetime` methods on each of the objects.  
Let's first try with getting the day of week, for which we can use

```python
df["Caught date"].dt.day_of_week
```

Let's take a look at the first few elements with `head`.  
Note that 0 here means Monday, and 6 means Sunday.


In [ ]:
df1["Caught date"].dt.day_of_week.head()

We can now use `groupby` with the this transformed colum.  
Let's count how many turtles were caught on each of the weekdays.  

In [ ]:
df1["Caught date"].groupby(df1["Caught date"].dt.day_of_week).count()


Let's now add a meaningful index to this

In [ ]:
s = df1["Species"].groupby(df1["Caught date"].dt.day_of_week).count()
s.index = [  # type: ignore
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday",
]
s


### Exercise: weekday of catch

Using the series above, create a bar plot to show how many turtles were caught on each day of the week

In [ ]:
s = df1["Species"].groupby(df1["Caught date"].dt.day_of_week).count()
s.index = [  # type: ignore
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday",
]



## Groupby: multiple

We can also use combinations of selections, like for example each month of each year.

In [ ]:
(
    df1["Species"]
    .groupby([df1["Caught date"].dt.year, df1["Caught date"].dt.month])  # type: ignore
    .value_counts()
)


As you can see, the index is now a combination of both the year and the month, called a [MultiIndex](https://pandas.pydata.org/docs/user_guide/advanced.html#hierarchical-indexing-multiindex). These can be a bit annoying to work with, so we are going to combine both levels of that multi-index.

First, we are going to unstack the index, which will pivot one level of the index

In [ ]:
(
    df1["Species"]
    .groupby([df1["Caught date"].dt.year, df1["Caught date"].dt.month])  # type: ignore
    .value_counts()
    .unstack()
)


Then we are going to replace the index with a new, custom index

In [ ]:
# When where which species caught?
s = (
    df1["Species"]
    .groupby([df1["Caught date"].dt.year, df1["Caught date"].dt.month])  # type: ignore
    .value_counts()
    .unstack()
)
s.index = s.index.map(lambda xy: f"{xy[0]}-{xy[1]:02}")
s


Lastly, we are going to replace the `NaN` (not a number) entries with zeros.

In [ ]:
s = (
    df1["Species"]
    .groupby([df1["Caught date"].dt.year, df1["Caught date"].dt.month])  # type: ignore
    .value_counts()
    .unstack()
    .fillna(0)
)
s.index = s.index.map(lambda xy: f"{xy[0]}-{xy[1]:02}")
s


### Exercise: turtle distribution by month (absolute)

Using a bar plot, show how many of each turtle species were caught each month.

In [ ]:
s = (
    df1["Species"]
    .groupby([df1["Caught date"].dt.year, df1["Caught date"].dt.month])  # type: ignore
    .value_counts()
    .unstack()
    .fillna(0)
)
s.index = s.index.map(lambda xy: f"{xy[0]}-{xy[1]:02}")

### Exercise: turtle distribution by month (relative)

In the last exercise you showed the absolute number of each turtle species per month.  
Now, plot the *relative* amount of each turtle species per month by summing up the total number of turtles *per month* and then dividing the number of each species by the respective sum.  
Use a stacked bar plot or an area plot for this task.

In [ ]:
s = (
    df1["Species"]
    .groupby([df1["Caught date"].dt.year, df1["Caught date"].dt.month])  # type: ignore
    .value_counts()
    .unstack()
    .fillna(0)
)
s.index = s.index.map(lambda xy: f"{xy[0]}-{xy[1]:02}")




## Hands-on learning: trc data set

This next data set contains data about the admission and treatment of caught turtles.  
You will perform an analysis of which diagnoses lead to which outcome.  

In [ ]:
df2 = (
    pd.read_csv(Path("data") / "trc.csv")
    .set_index("Interaction ID")
    .rename(
        columns={
            "Admission  Outcome": "Admission Outcome",
            "Admission  Outcome Date": "Admission Outcome Date",
        }
    )
)
df2["Admission Date"] = pd.to_datetime(df2["Admission Date"], format=r"%m/%d/%y")
df2["Admission Outcome Date"] = pd.to_datetime(
    df2["Admission Outcome Date"], format=r"%m/%d/%y"
)
df2.head()


### Exercise

To get a feeling of how many turtles can be released, have to be euthanised or died during the treatment, count the different admission outcomes. 

### Exercise 

To get a feeling for the prevalence of each condition, count the values of all admission diagnoses.

### Exercise

Group the data by the admission diagnosis and then count how often each admission outcome can be observed.  

Plot your results as a bar plot

### Exercise

Repeat the previous task, but plot the outcome relative to the respective number of admission diagnoses.  

## Further learning

Documentation

- [numpy documentation](https://numpy.org/doc/stable/user/index.html#user)
- [pandas documentation](https://pandas.pydata.org/docs/user_guide/index.html)
- [matplotlib documentation](https://matplotlib.org/stable/index.html)

Further packages

- [seaborn](https://seaborn.pydata.org/): package built on top of matplotlib for statistical plots
- [SciPy](https://docs.scipy.org/doc/scipy/tutorial/index.html#user-guide): advanced scientific computing library
- [statsmodels](https://www.statsmodels.org/stable/index.html): statistical models
- [scikit-learn](https://scikit-learn.org/stable/): machine learning library
- [PyTorch](https://pytorch.org/docs/stable/index.html): deep learning library
- [tensorflow](https://www.tensorflow.org/tutorials): deep learning library
- [Keras](https://keras.io/): deep learning library 
- [aesara](https://github.com/aesara-devs/aesara) (used to be Theano): symbolic maths on multi-dimensional arrays
- [JAX](https://github.com/google/jax): Composable transformations of Python & numpy on GPUs 

Books

- [Jake VanderPlas - Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/)
- [Wes McKinnery - Python for Data Analysis](https://wesmckinney.com/book/)